In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

# 1. 장치 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 클래스 수 정의 (사용자 정의 trainset의 클래스 수에 맞게 설정)
num_classes = 10  # 예: 클래스가 10개인 데이터셋을 가정

# 3. ViT 전처리기 설정
#    Hugging Face에서 제공하는 ViT 전처리기를 사용하여 데이터 전처리 기준(mean, std 등)을 자동으로 불러옴
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# 4. 데이터 변환 설정
#    모든 이미지를 224x224 크기로 조정하고, ViT 전처리 기준에 맞춰 정규화
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# 5. 사용자 정의 데이터셋 불러오기
#    새로운 경로 설정 및 전처리를 적용하여 데이터셋을 생성
trainset_path = 'C:/Users/jongcheol/OneDrive/바탕 화면/Semester2/train_data'
trainset = datasets.ImageFolder(root=trainset_path, transform=transform)
labels = np.array([label for _, label in trainset.imgs])  # 각 이미지의 레이블을 numpy 배열로 추출 (StratifiedKFold에 필요)

# 6. 5-Fold Cross Validation 설정
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # 5개 폴드로 데이터셋을 나누고, 레이블 분포가 비슷하도록 Stratified KFold 사용
fold_accuracies = []  # 각 폴드의 정확도를 저장할 리스트

# 7. Cross Validation 학습 및 평가
for fold, (train_idx, val_idx) in enumerate(kf.split(np.zeros(len(labels)), labels)):
    print(f"Fold {fold + 1} 시작")

    # Fold별 데이터셋 생성
    train_subset = Subset(trainset, train_idx)
    val_subset = Subset(trainset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=2)

    # 8. Vision Transformer 모델 초기화
    #    Hugging Face의 ViT 모델을 불러와 num_classes에 맞게 최종 분류 계층을 설정
    model = ViTForImageClassification.from_pretrained(
        'google/vit-base-patch16-224-in21k',
        num_labels=num_classes
    ).to(device)

    # 9. 손실 함수 및 옵티마이저 설정
    #    다중 클래스 분류를 위한 CrossEntropyLoss와 Adam 옵티마이저 사용
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=2e-5)

    # 10. 학습 루프
    num_epochs = 3  # 예시로 3에포크 (실제 학습에서는 더 큰 값으로 설정 가능)
    for epoch in range(num_epochs):
        model.train()  # 학습 모드 설정
        running_loss = 0.0  # 에포크 손실 초기화
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # 데이터와 레이블을 장치로 이동

            # 순전파, 손실 계산, 역전파, 옵티마이저 업데이트
            optimizer.zero_grad()
            outputs = model(images).logits  # 모델 예측
            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트

            running_loss += loss.item()  # 배치 손실을 누적하여 에포크 손실 계산

        # 에포크 종료 후 평균 손실 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

    # 11. 검증 루프
    model.eval()  # 평가 모드 설정
    val_labels = []  # 실제 레이블을 저장할 리스트
    val_preds = []  # 예측 레이블을 저장할 리스트

    with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않음
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images).logits  # 모델 예측
            _, preds = torch.max(outputs, 1)  # 예측 레이블 추출
            val_labels.extend(labels.cpu().numpy())  # 실제 레이블 추가
            val_preds.extend(preds.cpu().numpy())  # 예측 레이블 추가

    # 12. 폴드 정확도 계산 및 저장
    fold_accuracy = accuracy_score(val_labels, val_preds)  # 정확도 계산
    fold_accuracies.append(fold_accuracy)  # 폴드별 정확도 저장
    print(f"Fold {fold + 1} Accuracy: {fold_accuracy * 100:.2f}%")

# 13. 5-Fold 평균 정확도 출력
print(f"Average 5-Fold Accuracy: {np.mean(fold_accuracies) * 100:.2f}%")

Fold 1 시작


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/3], Loss: 1.8896
Epoch [2/3], Loss: 0.9446
Epoch [3/3], Loss: 0.4807
Fold 1 Accuracy: 94.92%
Fold 2 시작


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/3], Loss: 1.8616
Epoch [2/3], Loss: 0.9122
Epoch [3/3], Loss: 0.4578
Fold 2 Accuracy: 95.18%
Fold 3 시작


...

Epoch [1/3], Loss: 1.8768
Epoch [2/3], Loss: 0.9296
Epoch [3/3], Loss: 0.4778
Fold 3 Accuracy: 94.26%
Fold 4 시작


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/3], Loss: 1.8307
Epoch [2/3], Loss: 0.8999
Epoch [3/3], Loss: 0.4703
Fold 4 Accuracy: 97.00%
Fold 5 시작


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/3], Loss: 1.8618
Epoch [2/3], Loss: 0.9069
Epoch [3/3], Loss: 0.4677
Fold 5 Accuracy: 95.69%
Average 5-Fold Accuracy: 95.41%


### 예측

In [12]:
import os
import pandas as pd
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

# 테스트 데이터 경로 설정
test_data_path = 'C:/Users/jongcheol/OneDrive/바탕 화면/Semester2/test_data'

# 테스트 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 테스트 데이터셋 클래스 정의 (이미지 폴더 구조가 없을 경우)
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path).convert("RGB")  # RGB로 변환 (흑백 방지)

        if self.transform:
            image = self.transform(image)
        
        return image, img_name

# 테스트 데이터셋 및 DataLoader 생성
test_dataset = TestDataset(test_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# train 데이터셋 로드하여 클래스명-인덱스 매핑 가져오기
idx_to_class = {v: k for k, v in trainset.class_to_idx.items()}  # 인덱스 -> 클래스명 매핑 생성


In [13]:
# 모델을 평가 모드로 설정
model.eval()
predictions = []

# 테스트 데이터 예측 수행
with torch.no_grad():
    for images, img_names in test_loader:
        images = images.to(device)
        outputs = model(images).logits
        _, preds = torch.max(outputs, 1)  # 예측된 클래스 인덱스 추출

        # 결과 저장 (이미지 파일명과 예측된 클래스명)
        for img_name, pred in zip(img_names, preds.cpu().numpy()):
            class_name = idx_to_class[pred]  # 예측된 인덱스를 클래스명으로 변환
            predictions.append((img_name, class_name))

# 예측 결과를 DataFrame으로 변환
pred_df = pd.DataFrame(predictions, columns=["filename", "label"])

# 결과를 CSV 파일로 저장
output_path = "ViT_predictions.csv"
pred_df.to_csv(output_path, index=False)
print(f"예측 결과가 '{output_path}' 파일에 클래스명으로 저장되었습니다.")

예측 결과가 'ViT_predictions.csv' 파일에 클래스명으로 저장되었습니다.
